In [13]:
import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error


In [4]:
df = pd.read_parquet('../data/yellow_tripdata_2023-01.parquet')
print(f"Number of columns for January: {df.shape[1]}")

df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

print(f"Standard deviation for column duration: {df['duration'].std()}")

print(f"Fraction of the records left: {((df.duration >= 1) & (df.duration <= 60)).mean()}")

df = df[(df.duration >= 1) & (df.duration <= 60)]


Number of columns for January: 19
Standard deviation for column duration: 42.594351241920904
Fraction of the records left: 0.9812202822125979


In [15]:
categorical = ['PULocationID', 'DOLocationID']

df[categorical] = df[categorical].astype(str)

train_dicts = df[categorical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)
print(f"The dimensionality of the matrix: {X_train.shape[1]}")

target = 'duration'
y_train = df[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

print(f"RMSE on train: {mean_squared_error(y_train, y_pred, squared=False)}")

The dimensionality of the matrix: 515
RMSE on train: 7.649261027792376


In [1]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [17]:
df_val = read_dataframe('../data/yellow_tripdata_2023-02.parquet')

In [20]:
dv = DictVectorizer()

val_dicts = df_val[categorical].to_dict(orient='records')
X_val = dv.fit_transform(val_dicts)

target = 'duration'
y_val = df_val[target].values

y_pred = lr.predict(X_val)

root_mean_squared_error(y_val, y_pred)

ValueError: X has 514 features, but LinearRegression is expecting 515 features as input.